**Mount drive to colab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Unzipping the dataset**

In [1]:
!unzip /content/drive/MyDrive/dataset.zip

**OBJECT DETECTION**


**import necessary libraries**

In [1]:
import os
import torch
import torchvision
from torchvision import transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import cv2

**Main function**

In [ ]:
def main():

    model = get_model()
    dataset_dir = '/content/dataset'
    process_dataset(dataset_dir, model, get_transform())

if __name__ == "__main__":
    main()

**Loading the pre-existing model**

In [ ]:
# Faster R-CNN model
def get_model():

    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

**Transformation(converting images to tensors)**

In [ ]:
# Function to transform input image
def get_transform():
    transform = T.Compose([
        T.ToTensor(),  # Convert PIL image or numpy array to a tensor
    ])
    return transform

**Performing inference**

In [ ]:
# Function to perform inference on a single image
def detect_objects(image_path, model, transform):

    image = Image.open(image_path).convert("RGB")

    # Apply the transformation to the image
    image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

    # Perform the inference
    with torch.no_grad():
        outputs = model(image_tensor)

    return outputs

**visualizing**

In [ ]:
def visualize_results(image_path, outputs, threshold=0.5):
    # Load the image using OpenCV
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Get the bounding boxes and labels
    boxes = outputs[0]['boxes'].cpu().numpy()
    scores = outputs[0]['scores'].cpu().numpy()

    # Draw the boxes on the image
    for i, box in enumerate(boxes):
        if scores[i] > threshold:
            # Convert box coordinates to integers
            box = box.astype(int)
            cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), color=(0, 255, 0), thickness=2)

    # Save the image with detections
    result_image_path = os.path.join('results', os.path.basename(image_path))
    cv2.imwrite(result_image_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))


**Function to process a directory of images and saving the result in results directory**

In [ ]:
def process_dataset(dataset_dir, model, transform):
    # Create a results directory if it doesn't exist
    if not os.path.exists('results'):
        os.makedirs('results')

    # Iterate over all images in the directory
    for image_name in os.listdir(dataset_dir):
        image_path = os.path.join(dataset_dir, image_name)

        if image_path.endswith(('.png', '.jpg', '.jpeg')):
            print(f"Processing {image_name}...")
            outputs = detect_objects(image_path, model, transform)
            visualize_results(image_path, outputs)

**ANALYZING THE FRQUENCY OF OBJECTS DETECTED**

**import necessary libraries**

In [2]:
import os
import torch
import torchvision
from torchvision import transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from collections import defaultdict

**main function**

In [ ]:
def main():
    model = get_model()
    dataset_dir = '/content/dataset'
    object_catalog = process_dataset(dataset_dir, model, get_transform())

    total_objects = sum(object_catalog.values())
    print(f"Total objects detected: {total_objects}")
    for obj, count in object_catalog.items():
        print(f"{obj}: {count} ({(count/total_objects)*100:.2f}%)")

if __name__ == "__main__":
    main()


**COCO class labels**

In [ ]:
# COCO class labels
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant',
    'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A', 'handbag',
    'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',
    'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli',
    'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A',
    'dining table', 'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book', 'clock',
    'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]


**model loading and image transformation for object detection**

In [ ]:
# Function to get the pretrained Faster R-CNN model
def get_model():
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    return model

# Function to transform input image
def get_transform():
    transform = T.Compose([
        T.ToTensor(),
    ])
    return transform
# Function to perform inference on a single image
def detect_objects(image_path, model, transform):
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0)
    with torch.no_grad():
        outputs = model(image_tensor)
    return outputs

**Process the data and cataloging the objects**

In [ ]:
def visualize_and_catalog(image_path, outputs, object_catalog, threshold=0.5):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    boxes = outputs[0]['boxes'].cpu().numpy()
    scores = outputs[0]['scores'].cpu().numpy()
    labels = outputs[0]['labels'].cpu().numpy()

    for i, box in enumerate(boxes):
        if scores[i] > threshold:
            # Convert box coordinates to integers
            box = box.astype(int)
            cv2.rectangle(image, (box[0], box[1]), (box[2], box[3]), color=(0, 255, 0), thickness=2)
            label = labels[i]
            object_catalog[COCO_INSTANCE_CATEGORY_NAMES[label]] += 1

    result_image_path = os.path.join('results', os.path.basename(image_path))
    cv2.imwrite(result_image_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))


**Function to process a directory of images and catalog objects**

In [ ]:
def process_dataset(dataset_dir, model, transform):
    if not os.path.exists('results'):
        os.makedirs('results')

    object_catalog = defaultdict(int)

    for image_name in os.listdir(dataset_dir):
        image_path = os.path.join(dataset_dir, image_name)
        if image_path.endswith(('.png', '.jpg', '.jpeg')):
            print(f"Processing {image_name}...")
            outputs = detect_objects(image_path, model, transform)
            visualize_and_catalog(image_path, outputs, object_catalog)

    return object_catalog